In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sys
from pathlib import Path

# EDA

In [ ]:
# notebooks

# ROOT apunta a la carpeta raíz del proyecto
ROOT = Path().resolve().parent
sys.path.append(str(ROOT))

# 2️⃣ Importar funciones desde src
from src.dataset import load_data

# 3️⃣ Cargar datos
df = load_data()

# 4️⃣ Mostrar las primeras filas

df.columns


In [ ]:
def split_train_test_by_rows(df, n_test):
    if n_test <= 0 or n_test >= len(df):
        raise ValueError("n_test debe ser mayor que 0 y menor que el número de filas del DataFrame")
    
    test_df = df.tail(n_test)
    train_df = df.iloc[:-n_test]
    
    return train_df, test_df

In [ ]:
train_df, test_df = split_train_test_by_rows(df, 9158)
num_cols = train_df.columns[2:-1]  
train_df.loc[:, num_cols] = train_df.loc[:, num_cols].apply(pd.to_numeric, errors='coerce')
train_df.head()


In [ ]:
df.describe().transpose()

In [ ]:
df.isnull().mean()

In [ ]:
# Número de observaciones duplicadas
df.duplicated().sum()

In [ ]:
# Tuneamos los colores para que sean más intuitivos
custom_palette = {
    0: "grey",
    1: "red"
}

# Pairplot:
# Funcion util (cuando tengamos pocas variables) para entender las relaciones entre las covariables con la respuesta y entre las propias variables

sns.pairplot(train_df[['Time', 'CellName', 'PRBUsageUL', 'PRBUsageDL', 'meanThr_DL',
       'meanThr_UL', 'maxThr_DL', 'maxThr_UL', 'meanUE_DL', 'meanUE_UL',
       'maxUE_DL', 'maxUE_UL', 'maxUE_UL+DL', 'Unusual']], hue = "Unusual", palette=custom_palette)

In [ ]:
plt.style.use('ggplot')
(df['Unusual']).hist()

df['Unusual'].value_counts()

In [ ]:
#CORRELACIÓN
corr_matrix = train_df.iloc[:,2:-1].corr()
print(corr_matrix)
sns.heatmap(abs(train_df.iloc[:,2:-1].corr()), cmap = "rocket_r")

## Analisis de correlación

-maxUE_DL, maxUE_DL y maxUE_UL+DL muy correlados

-maxUE_DL, maxUE_DL y maxUE_UL+DL muy correlados con PRBusageUL (sobretodo) y también con PRBusageDL

In [ ]:
train_df.iloc[:,:-1].plot(kind='box', subplots=True, layout=(4, 3), figsize=(12, 8), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [ ]:
# Lista con las observaciones que son un outilier en alguna de las covariables
mask = pd.Series(True, index=df.index)

for col in train_df.columns[2:-1]:
    # Cuartiles
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)

    # Rango intercuartilico
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Actualizamos mask: True solo si el valor está dentro de los límites
    mask &= train_df[col].between(lower_bound, upper_bound)

# Observaciones originales
print(f"Número de filas originales: {len(train_df)}")

# Aplicamos mask para eliminar filas con al menos un outlier
train_df_sinOutliers = train_df[mask]

# Resultado tras eliminar outliers
print(f"Número de filas sin outliers: {len(train_df_sinOutliers)}")

In [ ]:
train_df_sinOutliers.iloc[:,:-1].plot(kind='box', subplots=True, layout=(4, 3), figsize=(12, 8), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import skew

# Calculamos la asimetría
skewness = train_df_sinOutliers.select_dtypes(include='number').apply(skew)

# Ordenamos las variables por asimetría
print(skewness.sort_values(ascending=False))



In [ ]:
for col in train_df_sinOutliers.columns[2:-2]:
    # Calculamos la skewness
    sk = skew(train_df_sinOutliers[col])
    if sk > 1:
        train_df_sinOutliers[col] = np.log1p(train_df_sinOutliers[col])  # log(x + 1) para evitar log(0)
    elif sk < -1:
        max_val = train_df_sinOutliers[col].max()
        train_df_sinOutliers[col] = np.log1p(max_val + 1 - train_df_sinOutliers[col])  # reflejar + log

In [ ]:
from scipy.stats import skew

# Calculamos la asimetría
skewness = train_df_sinOutliers.select_dtypes(include='number').apply(skew)

# Ordenamos las variables por asimetría
print(skewness.sort_values(ascending=False))


### Situación actual:
train_df_sinOutliers -> train dataset, sin outliers y con asimetrías transformadas. 

train_df -> train dataset sin procesar

*Comprobar con que modelos se comporta mejor o peor cada uno, con los más simples (regresión logistica, KNN debería funcionar mejor el transformado. Puede ser que el completo de buenos resultados en modelos complejos capaces de captar ese tipo de patrones)

In [ ]:
train_df_sinOutliers.iloc[:,:-1].plot(kind='box', subplots=True, layout=(4, 3), figsize=(12, 8), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

# ENTRENAMIENTO MODELOS

## Escalado z-score

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_df.iloc[:,2:-1])

df_escalado = train_df

df_escalado.iloc[:,2:-1] = scaler.transform(df_escalado.iloc[:,2:-1])

df_escalado_procesado = train_df_sinOutliers

df_escalado_procesado.iloc[:,2:-1] = scaler.transform(df_escalado_procesado.iloc[:,2:-1])

### escalados

df_escalado_procesado -> sin outliers y simetrico

df_escalado -> original sin procesar pero si escalado

test_df_escalado -> test escalado (con scaler del train, para no contaminar con datos de test, y sin procesar ni limpiar ouliers) (NO tengo las etiquetas del test, la competición kaggle está cerrada. Usaré el test de train para comprobar la precisión, sacando subconjunto de validación)

## Modelo regresión logistica

In [ ]:
from sklearn.model_selection import train_test_split

#Sin preprocesar (con outliers y distribuciones asimetricas)

X_train, X_val, y_train, y_val = train_test_split(df_escalado.drop(['Time', 'CellName','Unusual'],axis=1),
                                                    df_escalado['Unusual'], test_size=0.30,
                                                    random_state=101)

filas_con_nan = X_train[X_train.isna().any(axis=1)]

# Mostrar las filas con NaN
print(f"Filas con NaN en X_train ({len(filas_con_nan)} de {len(X_train)}):")


Tenemos 67 de 25832 filas con valores nulos en maxUE. Se pueden eliminar o imputar.

1. Eliminando:

In [ ]:
# Eliminar NaN
X_train_sin_Nan = X_train.dropna()
y_train_sin_Nan = y_train.loc[X_train_sin_Nan.index]
X_val_sin_Nan = X_val.dropna()
y_val_sin_Nan = y_val.loc[X_val_sin_Nan.index]

In [ ]:
print(X_train['maxUE_DL'].mean())
print(X_train['maxUE_UL'].mean())
print(X_train['maxUE_UL+DL'].mean())

#de momento eliminamos y ya

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()

logmodel.fit(X_train_sin_Nan,y_train_sin_Nan)

In [ ]:
predictions = logmodel.predict(X_val_sin_Nan)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_val_sin_Nan,predictions))

print(confusion_matrix(y_val_sin_Nan,predictions))

Regresión logistica, con los datos sin preprocesar es malo, con un F1 de 0.42. El modelo está clasificando como 0 (comportamiento normal) con demasiada frecuencia, tiene 0.00423 de recall para la clase 1, es decir, de las que son 1 solo clasifica bien 0.04%. 
El modelo Ha sobreajustado a la clase 0 al ser mucho más numerosa y no añadir ninguna tecnica de control. 

## Sin outliers y con transofrmaciones para simetría

In [ ]:
#preprocesando

X_train_preprocesado, X_val_preprocesado, y_train, y_val = train_test_split(df_escalado_procesado.drop(['Time', 'CellName','Unusual'],axis=1),
                                                    df_escalado_procesado['Unusual'], test_size=0.30,
                                                    random_state=101)

In [ ]:
# Eliminar NaN
X_train_pre_sin_Nan = X_train_preprocesado.dropna()
y_train_sin_Nan = y_train.loc[X_train_pre_sin_Nan.index]
X_val_pre_sin_Nan = X_val_preprocesado.dropna()
y_val_sin_Nan = y_val.loc[X_val_pre_sin_Nan.index]

In [ ]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()

logmodel.fit(X_train_pre_sin_Nan,y_train_sin_Nan)

In [ ]:
predictions_pre = logmodel.predict(X_val_pre_sin_Nan)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_val_sin_Nan,predictions_pre))

print(confusion_matrix(y_val_sin_Nan,predictions_pre))

Al simplificar los datos, la regresión logística se comporta mejor. Sin embargo, los resultados siguen siendo malos, F1 de 0.49

### Prueba con mayor control en el modelo. 
1. Balancear numero de muestras en las clases.
2. Modificar umbral de decisión para predecir más 1 (inusuales). Estó mejorará el recall, a cambio de precisión.

In [ ]:
#seguimos con el preprocesado que tenia ligeramente mejor comportamiento

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(class_weight='balanced') #asigna pesos automaticamente a las muestras de cada tipo de forma inversmanete proporcional a su frecuencia en el dataset. Así se compensa la falta de presencia de muestras tipo 1 (comportamiento inusual)

logmodel.fit(X_train_pre_sin_Nan,y_train_sin_Nan)

In [ ]:
predictions_balanceado = logmodel.predict(X_val_pre_sin_Nan)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_val_sin_Nan,predictions_balanceado))

print(confusion_matrix(y_val_sin_Nan,predictions_balanceado))

mejora significativamente el rendimiento, pasamos a predecir de 13 a 1819 clases tipo 1 correctamente. La recall de la clase 1 aumenta de 0.09 a 0.71. Ahora el F1 score es de o.61

Vamos a estudiar también la ROC y su AUC. Ayudará también a ver si merece la pena ajustar el umbral de decisión o no.

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Obtener probabilidades de la clase 1
y_proba = logmodel.predict_proba(X_val_pre_sin_Nan)[:, 1]

# Calcular curva ROC
fpr, tpr, umbrales = roc_curve(y_val_sin_Nan, y_proba)
roc_auc = auc(fpr, tpr)

# Graficar
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Obtener TPR, FPR y umbrales
fpr, tpr, umbrales = roc_curve(y_val_sin_Nan, y_proba)

# Método 1: Distancia al punto (0, 1)
distancias = np.sqrt((fpr - 0)**2 + (tpr - 1)**2)
mejor_idx = np.argmin(distancias)
mejor_umbral_1 = umbrales[mejor_idx]

# Método 2: Índice de Youden
youden_index = tpr - fpr
mejor_idx_2 = np.argmax(youden_index)
mejor_umbral_2 = umbrales[mejor_idx_2]

print(f"Mejor umbral (distancia): {mejor_umbral_1:.3f}")
print(f"Mejor umbral (Youden): {mejor_umbral_2:.3f}")

In [ ]:
y_pred_custom = (y_proba > 0.54).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_val_sin_Nan, y_pred_custom))
print(confusion_matrix(y_val_sin_Nan, y_pred_custom))

Cómo se quiere maximizar el F1 nos quedamos con esta ultima versión. Modificando el umbral, estamos empeorando el recall de la clase 1, vamos a detectar menos porcentaje de comportamientos inusuales, pero vamos a tener menos falsos positivos (clasificados como inusual cuando son usuales). 

Una última prueba para mejorar el comportamiento del modelo de regresión logistica será gestionar la multicolinealidad. Al principio, en el EDA, se vió como había variables muy correladas.

In [ ]:
#Las que tenían mucha correlación eran 'maxUE_DL', 'maxUE_UL', 'maxUE_UL+DL'
#tambien 'meanThr_DL' con 'PRBUsageDL'

X_train_reduced = X_train_pre_sin_Nan.drop(['maxUE_DL','maxUE_UL','meanThr_DL'], axis=1)
x_val_reduced = X_val_pre_sin_Nan.drop(['maxUE_DL','maxUE_UL','meanThr_DL'], axis=1)

x_val_reduced.columns


In [ ]:
#seguimos con el preprocesado que tenia ligeramente mejor comportamiento

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(class_weight='balanced') #asigna pesos automaticamente a las muestras de cada tipo de forma inversmanete proporcional a su frecuencia en el dataset. Así se compensa la falta de presencia de muestras tipo 1 (comportamiento inusual)

logmodel.fit(X_train_reduced,y_train_sin_Nan)

In [ ]:
predictions_balanceado = logmodel.predict(x_val_reduced)

In [ ]:
y_proba = logmodel.predict_proba(x_val_reduced)[:, 1]

y_pred_custom = (y_proba > 0.54).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_val_sin_Nan, y_pred_custom))
print(confusion_matrix(y_val_sin_Nan, y_pred_custom))

No ha afectado eliminar las variables mas correladas

## Modelo más complejo - Random Forest

voy a probar este modelo mas complejo para intentar mejorarar el F1. Si la relación entre target y variables es no lineal, este funcionará mejor

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Modelo con parámetros iniciales
rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  
    random_state=42,
    n_jobs=-1 
)

voy a probar con los datos sin preprocesar, al ser un modelo más complejo puede que capte bien esos patrones

In [ ]:
# Entrenar
rf_model.fit(X_train_sin_Nan, y_train_sin_Nan)

# Predecir probabilidades (para ajustar umbral después)
y_proba_rf = rf_model.predict_proba(X_val_sin_Nan)[:, 1]

# Predecir con umbral por defecto (0.5)
y_pred_rf = rf_model.predict(X_val_sin_Nan)

In [ ]:
print("Random Forest - Resultados:")
print(classification_report(y_val_sin_Nan, y_pred_rf))
print("Matriz de confusión:")
print(confusion_matrix(y_val_sin_Nan, y_pred_rf))

mejora considerablemente la calidad de las predicciones del modelo usando Random Forest. Veamos si es mejor sin outliers y distribuciones simetricas

In [ ]:
# Entrenar
rf_model.fit(X_train_pre_sin_Nan, y_train_sin_Nan)

# Predecir probabilidades (para ajustar umbral después)
y_proba_rf = rf_model.predict_proba(X_val_pre_sin_Nan)[:, 1]

# Predecir con umbral por defecto (0.5)
y_pred_rf = rf_model.predict(X_val_pre_sin_Nan)

In [ ]:
print("Random Forest - Resultados:")
print(classification_report(y_val_sin_Nan, y_pred_rf))
print("Matriz de confusión:")
print(confusion_matrix(y_val_sin_Nan, y_pred_rf))

mejora la precisón eliminando outliers. Lo última sería intentar ajustar hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir parámetros a optimizar
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Búsqueda grid con validación cruzada
RF_grid_search = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=42),
    param_grid,
    cv=5,
    scoring='f1',  # Optimizar para F1-score
    n_jobs=-1
)

RF_grid_search.fit(X_train_pre_sin_Nan, y_train_sin_Nan)

# Mejores parámetros
print("Mejores parámetros:", RF_grid_search.best_params_)

In [ ]:
# Predecir probabilidades (para ajustar umbral después)
y_proba_rf = RF_grid_search.predict_proba(X_val_pre_sin_Nan)[:, 1]

# Predecir con umbral por defecto (0.5)
y_pred_rf = RF_grid_search.predict(X_val_pre_sin_Nan)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Obtener probabilidades de la clase 1
y_proba = RF_grid_search.predict_proba(X_val_pre_sin_Nan)[:, 1]

# Calcular curva ROC
fpr, tpr, umbrales = roc_curve(y_val_sin_Nan, y_proba)
roc_auc = auc(fpr, tpr)

# Graficar
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Obtener TPR, FPR y umbrales
fpr, tpr, umbrales = roc_curve(y_val_sin_Nan, y_proba)

# Método 1: Distancia al punto (0, 1)
distancias = np.sqrt((fpr - 0)**2 + (tpr - 1)**2)
mejor_idx = np.argmin(distancias)
mejor_umbral_1 = umbrales[mejor_idx]

# Método 2: Índice de Youden
youden_index = tpr - fpr
mejor_idx_2 = np.argmax(youden_index)
mejor_umbral_2 = umbrales[mejor_idx_2]

print(f"Mejor umbral (distancia): {mejor_umbral_1:.3f}")
print(f"Mejor umbral (Youden): {mejor_umbral_2:.3f}")

In [ ]:
y_pred_custom = (y_proba_rf > 0.389).astype(int)

from sklearn.metrics import classification_report
print("Random Forest - Resultados con umbral ajustado:")
print(classification_report(y_val_sin_Nan, y_pred_custom))
print("Matriz de confusión con umbral ajustado:")
print(confusion_matrix(y_val_sin_Nan, y_pred_custom))

print("Random Forest - Resultados:")
print(classification_report(y_val_sin_Nan, y_pred_rf))
print("Matriz de confusión:")
print(confusion_matrix(y_val_sin_Nan, y_pred_rf))